# Module import

In [1]:
from ETGEMs_function_LATCH import *

# Data initialization

In [2]:
import pandas as pd
import cobra

reaction_g0_file='./Basic Data_LATCH/reaction_g0_LATCH-1.0.txt'
metabolites_lnC_file = './Basic Data_LATCH/metabolites_lnC_LATCH-1.0.txt'
reaction_kcat_MW_file='./Basic Data_LATCH/ID_kcat_MW_file_LATCH-1.0.csv'

model=cobra.io.read_sbml_model('./Basic Data_LATCH/iML1515.xml')
cobra.manipulation.modify.convert_to_irreversible(model)

model.reactions.get_by_id('ATPM').bounds = (0.0,0.0)
model.reactions.get_by_id('EX_co2_e').bounds = (0.0, 0.0)
#model.reactions.get_by_id('ATPS4rpp').bounds = (0.0, 0.0)
model.reactions.get_by_id('HCO3E').bounds = (0.0, 0.0)#关闭碳酸之间转换


R=cobra.Reaction('NADPH_supply')
model.add_reaction(R)
R.build_reaction_from_string('nadp_c + h_c --> nadph_c',fwd_arrow="-->")
model.reactions.get_by_id('NADPH_supply').bounds = (4.0, 4.0)

R=cobra.Reaction('ATP_supply')
model.add_reaction(R)
R.build_reaction_from_string('adp_c + h_c + pi_c --> atp_c + h2o_c',fwd_arrow="-->")
model.reactions.get_by_id('ATP_supply').bounds = (4.0, 4.0)

###R1-4
R=cobra.Reaction('GlyR')#添加新反应,同内源反应GLYCLTDy
model.add_reaction(R)
R.build_reaction_from_string('glx_c + nadph_c + h_c --> glyclt_c + nadp_c',fwd_arrow="-->")
model.reactions.get_by_id('GlyR').bounds = (1.0, 1.0)

R=cobra.Reaction('GCS')
model.add_reaction(R)
R.build_reaction_from_string('glyclt_c + atp_c + coa_c + h2o_c --> glycolylcoa_c + amp_c + ppi_c + h_c',fwd_arrow="-->")
model.reactions.get_by_id('GCS').bounds = (1.0, 1.0)

R=cobra.Reaction('GCC')
model.add_reaction(R)
R.build_reaction_from_string('glycolylcoa_c + hco3_c + atp_c --> tartronylcoa_c + adp_c + pi_c + h_c',fwd_arrow="-->")
model.reactions.get_by_id('GCC').bounds = (1.0, 1.0)

R=cobra.Reaction('TCR')
model.add_reaction(R)
R.build_reaction_from_string('tartronylcoa_c + 2 nadph_c --> glyc__R_c + 2 nadp_c + coa_c',fwd_arrow="-->")
model.reactions.get_by_id('TCR').bounds = (1.0, 1.0)

#内源的R5-6
'''
GLYCK2
ENO
'''
model.reactions.get_by_id('ENO').bounds = (1.0, 1.0)
model.reactions.get_by_id('GLYCK2').bounds = (1.0, 1.0)
#model.reactions.get_by_id('PPC').bounds = (0.0, 0.0)

#R7-10
R=cobra.Reaction('PPC1')
model.add_reaction(R)
R.build_reaction_from_string('hco3_c + pep_c --> oaa_c + pi_c',fwd_arrow="-->")
model.reactions.get_by_id('PPC1').bounds = (1.0, 1.0)

R=cobra.Reaction('MDHII')#添加新反应
model.add_reaction(R)
R.build_reaction_from_string('h_c + nadph_c + oaa_c --> mal__L_c + nadp_c',fwd_arrow="-->")
model.reactions.get_by_id('MDHII').bounds = (1.0, 1.0)

R=cobra.Reaction('MTK')
model.add_reaction(R)
R.build_reaction_from_string('atp_c + mal__L_c + coa_c --> adp_c + pi_c + malylcoa_c',fwd_arrow="-->")
model.reactions.get_by_id('MTK').bounds = (1.0, 1.0)

R=cobra.Reaction('MCL')
model.add_reaction(R)
R.build_reaction_from_string('malylcoa_c --> accoa_c + glx_c',fwd_arrow="-->")
model.reactions.get_by_id('MCL').bounds = (1.0, 1.0)

#偶联反应-内源RS1-S5
'''
PPA
PTAr
ACKr
EX_ac_e
ADK1
'''
model.reactions.get_by_id('PPA').bounds = (1.0, 1.0)
model.reactions.get_by_id('PTAr').bounds = (1.0, 1.0)
#model.reactions.get_by_id('ACKr').bounds = (-1.0, -1.0)

model.reactions.get_by_id('ADK1').bounds = (1.0, 1.0)
model.reactions.get_by_id('EX_ac_e').bounds = (1.0, 1.0)

R=cobra.Reaction('Ex_h_c')
model.add_reaction(R)
R.build_reaction_from_string('--> h_c',fwd_arrow="-->")
model.reactions.get_by_id('Ex_h_c').bounds = (-1000.0, 1000.0)

R=cobra.Reaction('Ex_h_p')
model.add_reaction(R)
R.build_reaction_from_string('h_p -->',fwd_arrow="-->")
model.reactions.get_by_id('Ex_h_p').bounds = (-1000.0, 1000.0)


R=cobra.Reaction('EX_HCO3')
model.add_reaction(R)
R.build_reaction_from_string(' --> hco3_c',fwd_arrow="-->")
model.reactions.get_by_id('EX_HCO3').bounds = (0.0, 2.0)

model.reactions.get_by_id('EX_glc__D_e').bounds = (0.0, 0.0)
model.reactions.get_by_id('GLCtex_copy2').bounds = (0.0, 0.0)
model.reactions.get_by_id('GLCtex_copy1').bounds = (0.0, 0.0)

Concretemodel_Need_Data=Get_Concretemodel_Need_Data(reaction_g0_file,metabolites_lnC_file,model,reaction_kcat_MW_file)

unknown metabolite 'glycolylcoa_c' created
unknown metabolite 'tartronylcoa_c' created
unknown metabolite 'malylcoa_c' created


# 1.MDF calculation
Preset a lower growth rate, for example, when set "product_value = 0.3", and then get an MDF value (2.667); when set "product_value = 0.7074", and then get an MDF value (-14.571).

In [3]:
product_id = 'EX_ac_e'#
E_total = 10 #eaual to e_pool （0.228）* saturation（0.5）
substrate_name = 'EX_HCO3'
substrate_value = 2
product_value = 1
K_value=1249

B_value=MDF_Calculation(Concretemodel_Need_Data,product_value,product_id,substrate_name,substrate_value,K_value,E_total,'gurobi')
print("B value : " +str(B_value))

B value : 19.536636323256253


# 2.Maximum growth rate calculation
By taking the MDF above obtained as the lower bound of thermodynamic constraints.

In [4]:
obj_name='EX_ac_e'
obj_target='maximize'
E_total=10
#substrate_name='EX_glc__D_e_reverse'
substrate_name='EX_HCO3'
substrate_value=2
K_value=1249

max_product_under_mdf=Max_Growth_Rate_Calculation(Concretemodel_Need_Data,obj_name,obj_target,substrate_name,substrate_value,K_value,E_total,B_value,'gurobi')
print("Max product value : " +str(max_product_under_mdf))

Max product value : 1.0


# 3.Minimum enzyme cost calculation
By fixing the MDF and maximum growth rate above obtained.

In [5]:
product_id='EX_ac_e'
E_total=10 #eaual to e_pool （0.228）* saturation（0.5）
substrate_name='EX_HCO3'
substrate_value=2
product_value=max_product_under_mdf
K_value=1249
B_value=B_value
#B_value=0

min_E=Min_Enzyme_Cost_Calculation(Concretemodel_Need_Data,product_value,product_id,substrate_name,substrate_value,K_value,E_total,B_value,'gurobi')
print("Min enzyme amount : " +str(min_E))

Min enzyme amount : 0.0381198324710043


# 4.Minimum flux sum calculation（pFBA）
Used to simplify the output file below

In [6]:
product_id='EX_ac_e'
E_total=10 #eaual to e_pool （0.228）* saturation（0.5）
substrate_name='EX_HCO3'
substrate_value=2
product_value=max_product_under_mdf
K_value=1249
B_value=B_value
#B_value=B_0

[min_V,Concretemodel]=Min_Flux_Sum_Calculation(Concretemodel_Need_Data,product_value,product_id,substrate_name,substrate_value,K_value,E_total,B_value,'gurobi')
print("Min flux amount : " +str(min_V))

Min flux amount : 42.5


# 5.Pathway information output
It is used to extract the following various lists.

In [7]:
model=Concretemodel_Need_Data['model']
reaction_kcat_MW=Concretemodel_Need_Data['reaction_kcat_MW']
reaction_g0=Concretemodel_Need_Data['reaction_g0']
coef_matrix=Concretemodel_Need_Data['coef_matrix']
metabolite_list=Concretemodel_Need_Data['metabolite_list']
use_result = Get_Results_Thermodynamics(model,Concretemodel,reaction_kcat_MW,reaction_g0,coef_matrix,metabolite_list)
use_result = use_result[use_result['flux'] > 1e-10] 
use_result = use_result.sort_values(by = 'flux',axis = 0,ascending = False)
use_result["reaction"] = use_result.apply(lambda row: model.reactions.get_by_id(row.name).reaction, axis = 1)
use_result["gpr"] = use_result.apply(lambda row: model.reactions.get_by_id(row.name).gene_reaction_rule, axis = 1)
use_result.to_csv('./Analysis Result/LATCH/' + 'EX_ac_e' + str(round(max_product_under_mdf,3)) + '_' + str(round(B_value,3)) + '_' + str(round(min_E,3)) + '-LATCH-MDF-422.csv', sep=',', header=True, index=True,mode='w')

# 6.List extraction of candidate bottleneck reactions
Standard: the thermodynamic driving force (f) is equal to MDF value (B) above mentiond

In [8]:
use_result_tmp=use_result[use_result['f']>-1249]
use_result_select=use_result_tmp[abs(use_result_tmp['f']-B_value)<= 1249]
use_result_select.head(100)

,flux,z,f,enz,met_concentration,reaction,gpr
MCL,1.0,1.0,19.536636,0.000593,;accoa_c : 0.02000000000856292;malylcoa_c : 1....,malylcoa_c --> accoa_c + glx_c,
GLYCK2,1.0,1.0,19.536636,0.000304,;adp_c : 4.999999992621096e-07;h_c : 1.0;glyc_...,atp_c + glyc__R_c --> 2pg_c + adp_c + h_c,b3124
ACKr_reverse,1.0,1.0,19.536636,0.000007,;ac_c : 4.999999992621096e-07;adp_c : 4.999999...,actp_c + adp_c --> ac_c + atp_c,b1849 or b2296 or b3115
ENO,1.0,1.0,19.536636,0.000110,;h2o_c : 1.0;2pg_c : 0.00022333866889188492;pe...,2pg_c --> h2o_c + pep_c,b2779
ADK1,1.0,1.0,19.536636,0.000211,;adp_c : 4.999999992621096e-07;amp_c : 0.00109...,amp_c + atp_c --> 2.0 adp_c,b0474
MTK,1.0,1.0,19.536636,0.002252,;mal__L_c : 0.02000000000856292;coa_c : 2.2201...,atp_c + coa_c + mal__L_c --> adp_c + malylcoa_...,
GCC,1.0,1.0,26.139618,0.006419,;hco3_c : 0.02000000000856292;tartronylcoa_c :...,atp_c + glycolylcoa_c + hco3_c --> adp_c + h_c...,
GCS,1.0,1.0,19.536636,0.001764,;h2o_c : 1.0;ppi_c : 4.999999992621096e-07;h_c...,atp_c + coa_c + glyclt_c + h2o_c --> amp_c + g...,
GlyR,1.0,1.0,19.536636,0.000017,;nadph_c : 4.675575957122816e-06;h_c : 1.0;glx...,glx_c + h_c + nadph_c --> glyclt_c + nadp_c,
TCR,1.0,1.0,28.330711,0.026190,;tartronylcoa_c : 4.999999992621096e-07;nadph_...,2.0 nadph_c + tartronylcoa_c --> coa_c + glyc_...,


In [9]:
path_reac_list=list(use_result_select.index)

# 7.Determination of bottleneck reaction
Calculate the maximum thermodynamic driving force for reactions in above list, if the value is still equal to B, it is the bottleneck reaction.

In [10]:
import pandas as pd
import numpy as np
import datetime
from pyomo.environ import *
from pyomo.opt import SolverFactory
import pyomo.environ as pyo
from concurrent.futures import ProcessPoolExecutor, as_completed
starttime = datetime.datetime.now()
max_min_Df_list_fixed=pd.DataFrame()

with ProcessPoolExecutor() as executor:
    futures = {executor.submit(Get_Max_Min_Df_Complete,Concretemodel_Need_Data,eachreaction,'maximize',K_value,B_value,max_product_under_mdf,\
                               product_id,E_total,substrate_name,substrate_value,'gurobi'): eachreaction for eachreaction in path_reac_list}
    for future in as_completed(futures):
        tmp = future.result()
        for eachindex in tmp.index:
            print(eachindex,tmp.loc[eachindex,'max_value'])
            max_min_Df_list_fixed.loc[eachindex,'max_Df_complete']=tmp.loc[eachindex,'max_value']

endtime = datetime.datetime.now()
print (endtime - starttime)
#max_min_Df_list_fixed.to_csv('./Analysis Result/max_min_Df_complete_for_specific_reaction_trp_glc10.csv', sep=',', header=True, index=True,mode='w')
max_min_Df_list_fixed.to_csv('./Analysis Result/LATCH/max_min_Df_complete_for_all_reaction_for_ac_LATCH-MDF-422.csv', sep=',', header=True, index=True,mode='w')

GLYCK2 43.376253974926655
MCL 19.536640323256663
ACKr_reverse 19.536640323256865
ENO 31.128720981565007
GCC 47.63170630997992
MTK 19.536640323256524
GCS 37.18417331661825
GlyR 58.74754099336273
ADK1 37.184173316618335
TCR 123.5148879262606
PPC1 56.039617651670596
PPA 37.18417331661825
PTAr 19.536640323256467
MDHII 53.82872498156551
0:00:17.606481


In [11]:
max_min_Df_list_fixed=max_min_Df_list_fixed.sort_values(by='max_Df_complete',ascending = True)
max_min_Df_list_fixed.head(150)

,max_Df_complete
PTAr,19.536640
MTK,19.536640
MCL,19.536640
ACKr_reverse,19.536640
ENO,31.128721
GCS,37.184173
PPA,37.184173
ADK1,37.184173
GLYCK2,43.376254
GCC,47.631706


In [12]:
Bottleneck_reaction=max_min_Df_list_fixed[(max_min_Df_list_fixed['max_Df_complete']-B_value)<=0.001]
Bottleneck_reaction

,max_Df_complete
PTAr,19.53664
MTK,19.53664
MCL,19.53664
ACKr_reverse,19.53664


In [13]:
use_result_select.loc[Bottleneck_reaction.index[0],'met_concentration']

';accoa_c : 0.02000000000856292;coa_c : 2.2201325878783464e-06;actp_c : 4.278780280969315e-06;pi_c : 2.7015849431028562e-05'

# 8.List extraction of candidate limiting metabolites
Standard: involved in bottleneck reactions, except fo water (h2o_c) and protons (h_c).

In [14]:
Bottleneck_reaction_lsit=list(Bottleneck_reaction.index)
Bottleneck_reaction_met=[]
for rea in model.reactions:
    if rea.id in Bottleneck_reaction_lsit:
        #print(rea)
        for met in model.metabolites:
            try:
                rea.get_coefficient(met.id)  
            except:
                pass
            else:
                if met.id !='h_c' and met.id !='h2o_c':
                    Bottleneck_reaction_met.append(met.id)
                

Bottleneck_reaction_met=list(set(Bottleneck_reaction_met))
Bottleneck_reaction_met

['malylcoa_c',
 'accoa_c',
 'actp_c',
 'adp_c',
 'atp_c',
 'glx_c',
 'mal__L_c',
 'ac_c',
 'pi_c',
 'coa_c']

# 9.Determination of limitting metabolites
Calculate the maximum and minimum concentrations for metabolites in above list, if the two values are equal, it is the limiting metabolite.

In [15]:
import pandas as pd
import numpy as np
import datetime
from pyomo.environ import *
from pyomo.opt import SolverFactory
import pyomo.environ as pyo
from concurrent.futures import ProcessPoolExecutor, as_completed
starttime = datetime.datetime.now()
max_min_concentration_list_fixed = pd.DataFrame()

with ProcessPoolExecutor() as executor:
    futures = {executor.submit(Get_Max_Min_Met_Concentration,Concretemodel_Need_Data,eachmet,'maximize',K_value,B_value,\
        max_product_under_mdf,product_id,E_total,substrate_name,substrate_value,list(Bottleneck_reaction.index),'gurobi'): eachmet for eachmet in Bottleneck_reaction_met}
    for future in as_completed(futures):
        tmp = future.result()
        for eachindex in tmp.index:
            #print(eachindex,tmp.loc[eachindex,'max_value'])
            max_min_concentration_list_fixed.loc[eachindex,'max_concentration'] = tmp.loc[eachindex,'max_value']

with ProcessPoolExecutor() as executor:
    futures = {executor.submit(Get_Max_Min_Met_Concentration,Concretemodel_Need_Data,eachmet,'minimize',K_value,B_value,\
        max_product_under_mdf,product_id,E_total,substrate_name,substrate_value,list(Bottleneck_reaction.index),'gurobi'): eachmet for eachmet in Bottleneck_reaction_met}
    for future in as_completed(futures):
        tmp = future.result()
        for eachindex in tmp.index:
            #print(eachindex,tmp.loc[eachindex,'max_value'])
            max_min_concentration_list_fixed.loc[eachindex,'min_concentration'] = tmp.loc[eachindex,'min_value']
            
endtime = datetime.datetime.now()
print (endtime - starttime)
max_min_concentration_list_fixed.to_csv('./Analysis Result/LATCH/max_min_concentration_for_specific_metabolite_for_LATCH-MDF-422.csv', sep=',', header=True, index=True,mode='w')


0:00:27.069140


In [16]:
max_min_concentration_list_fixed

,max_concentration,min_concentration
accoa_c,-3.912023,-7.226305
malylcoa_c,-11.194376,-14.508658
atp_c,-3.912023,-14.508658
adp_c,-7.757823,-14.508658
glx_c,-14.508656,-14.508658
actp_c,-4.264065,-13.507809
ac_c,-14.508656,-14.508658
pi_c,-3.912023,-11.264444
coa_c,-9.061766,-14.508658
mal__L_c,-3.912023,-3.912025


In [17]:
Limiting_metabolite = max_min_concentration_list_fixed[(max_min_concentration_list_fixed['max_concentration'] - max_min_concentration_list_fixed['min_concentration']) <= 0.001]
Limiting_metabolite

,max_concentration,min_concentration
glx_c,-14.508656,-14.508658
ac_c,-14.508656,-14.508658
mal__L_c,-3.912023,-3.912025


# 10.List extraction of candidate key enzymes
Standard: The amount of enzyme usage was more than 0.0015 mg/gDW （above 1% of e_pool）

In [18]:
step5_file = pd.read_csv('./Analysis Result/LATCH/' + 'EX_ac_e' + str(round(max_product_under_mdf,3)) + '_' + str(round(B_value,3)) + '_' + str(round(min_E,3)) + '-LATCH-MDF-422.csv',index_col=0)
step5_file_sort = step5_file.sort_values(by='enz',ascending = False)
step5_file_sort.head(20)

,flux,z,f,enz,met_concentration,reaction,gpr
TCR,1.000,1.0,28.330711,0.026190,;tartronylcoa_c : 4.999999992621096e-07;nadph_...,2.0 nadph_c + tartronylcoa_c --> coa_c + glyc_...,NaN
GCC,1.000,1.0,26.139618,0.006419,;hco3_c : 0.02000000000856292;tartronylcoa_c :...,atp_c + glycolylcoa_c + hco3_c --> adp_c + h_c...,NaN
MTK,1.000,1.0,19.536636,0.002252,;mal__L_c : 0.02000000000856292;coa_c : 2.2201...,atp_c + coa_c + mal__L_c --> adp_c + malylcoa_...,NaN
GCS,1.000,1.0,19.536636,0.001764,;h2o_c : 1.0;ppi_c : 4.999999992621096e-07;h_c...,atp_c + coa_c + glyclt_c + h2o_c --> amp_c + g...,NaN
MCL,1.000,1.0,19.536636,0.000593,;accoa_c : 0.02000000000856292;malylcoa_c : 1....,malylcoa_c --> accoa_c + glx_c,NaN
GLYCK2,1.000,1.0,19.536636,0.000304,;adp_c : 4.999999992621096e-07;h_c : 1.0;glyc_...,atp_c + glyc__R_c --> 2pg_c + adp_c + h_c,b3124
ADK1,1.000,1.0,19.536636,0.000211,;adp_c : 4.999999992621096e-07;amp_c : 0.00109...,amp_c + atp_c --> 2.0 adp_c,b0474
ENO,1.000,1.0,19.536636,0.000110,;h2o_c : 1.0;2pg_c : 0.00022333866889188492;pe...,2pg_c --> h2o_c + pep_c,b2779
PPC1,1.000,1.0,34.639614,0.000102,;hco3_c : 0.02000000000856292;pep_c : 4.999999...,hco3_c + pep_c --> oaa_c + pi_c,NaN
PTAr,1.000,1.0,19.536636,0.000066,;accoa_c : 0.02000000000856292;coa_c : 2.22013...,accoa_c + pi_c --> actp_c + coa_c,b2297 or b2458
